In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '882432'

In [5]:
task = "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Retrieve'}}
----
Distance to neareast plan: 0.3283747434616089
{'Retrieve': {'messages': [HumanMessage(content="Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's", name='User')], 'session_id': '882432', 'previous_node': 'Retrieve', 'next': 'Modify', 'plan': None, 'task': "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's", 'code': None, 'function_detail': 'Text between the <statcast_documentation></statcast_documentation> tags is documentation for the statcast library.  Consult this section to confirm which attributes to pass into the statcast library.\n<statcast_documentation>\n# Statcast\n`statcast(start_dt=[yesterday\'s date], end_dt=None, team=None, verbose=True, parallel=True)`\n\nThe `statcast` function retrieves pitch-level statcast data for a given date or range or dates. \n\n## Returned data\nThis function returns a pandas `DataFra

In [6]:
task = "looks good"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Execute'}}
----
Known plan. Executing with Sonnet

Executing:
# Import required libraries
from pybaseball import statcast, statcast_pitcher, playerid_lookup
import pandas as pd
from sklearn.neighbors import NearestNeighbors

Executing:
start_date = '2019-06-01' 
end_date = '2019-06-07'

Executing:
verlander_id = playerid_lookup('verlander', 'justin').key_mlbam.iloc[0]
print(f"Justin Verlander's player ID is: {verlander_id}")

Result:
Gathering player lookup table. This may take a moment.
Justin Verlander's player ID is: 434378


Executing:
verlander_data = statcast_pitcher(start_date, end_date, player_id=verlander_id)
verlander_changes = verlander_data[verlander_data.pitch_type == 'CH']
print(f"Found {len(verlander_changes)} changeups for Verlander in this period")

Result:
Gathering Player Data
Found 4 changeups for Verlander in this period


Executing:
all_data = statcast(start_date, end_date, type='pitcher')
all_changes = all_data[all_data.pitch_type == 'CH

  0%|          | 0/7 [00:00<?, ?it/s]


Executing:
all_data = statcast(start_date, end_date)  
all_changes = all_data[all_data.pitch_type == 'CH']
print(f"Found {len(all_changes)} total changeups in this period")


100%|██████████| 7/7 [00:12<00:00,  1.84s/it]



Result:
This is a large query, it may take a moment to complete
Found 3020 total changeups in this period


Executing:
pitcher_averages = all_changes.groupby('pitcher')[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']].mean()
print(f"Calculated average changeup metrics for {len(pitcher_averages)} pitchers")

Result:
Calculated average changeup metrics for 253 pitchers


Executing:
verlander_avg = verlander_changes[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']].mean()
print("Verlander's average changeup metrics:")
print(verlander_avg)

Result:
Verlander's average changeup metrics:
release_speed          87.9500
release_spin_rate    1762.7500
pfx_x                  -1.2800
pfx_z                   0.7175
dtype: float64


Executing:
model = NearestNeighbors(n_neighbors=3)
model.fit(pitcher_averages)

distances, indices = model.kneighbors([verlander_avg])

similar_pitchers = pitcher_averages.iloc[indices[0]].index

print("The 3 pitchers with changeups most similar to Ver

In [7]:
task = "yes"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Memorize'}}
----
{'Memorize': {'messages': [HumanMessage(content="Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's", name='User'), AIMessage(content="1. Import the statcast, statcast_pitcher, and playerid_lookup functions from pybaseball:\n\n```python\nfrom pybaseball import statcast, statcast_pitcher, playerid_lookup\n```\n\n2. Define the start and end dates for the first week of June 2019:\n\n```python \nstart_date = '2019-06-01'\nend_date = '2019-06-07'\n```\n\n3. Look up Justin Verlander's MLBAM ID:\n\n```python\nverlander_id = playerid_lookup('verlander', 'justin').key_mlbam.iloc[0]\n```\n\n4. Get Verlander's statcast pitching data for the time period and filter to only changeups:\n\n```python\nverlander_data = statcast_pitcher(start_date, end_date, player_id=verlander_id)\nverlander_changes = verlander_data[verlander_data.pitch_type == 'CH']\n```\n\n5. Get statcast pitching data for all pitche